In [2]:
!uv pip install torchinfo
%load_ext autoreload
%autoreload 2

import os, sys
import torch
from torchinfo import summary
import matplotlib.pyplot as plt
# make sure notebook sees your code
sys.path.append(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Device:", device)

Audited 1 package in 11ms
Automatic pdb calling has been turned ON
Device: cuda


In [3]:
import yaml
from munch import Munch, munchify
import torch

# read your training config
cfg = munchify(yaml.safe_load(open("Configs/config_ja.yml")))

print(cfg.model_params)

Munch({'multispeaker': False, 'dim_in': 64, 'hidden_dim': 512, 'max_conv_dim': 512, 'n_layer': 3, 'n_mels': 80, 'n_token': 186, 'max_dur': 50, 'style_dim': 128, 'dropout': 0.2, 'decoder': Munch({'type': 'istftnet', 'resblock_kernel_sizes': [3, 7, 11], 'upsample_rates': [10, 6], 'upsample_initial_channel': 512, 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'upsample_kernel_sizes': [20, 12], 'gen_istft_n_fft': 20, 'gen_istft_hop_size': 5}), 'slm': Munch({'model': 'microsoft/wavlm-base-plus', 'sr': 16000, 'hidden': 768, 'nlayers': 13, 'initial_channel': 64}), 'diffusion': Munch({'embedding_mask_proba': 0.1, 'transformer': Munch({'num_layers': 3, 'num_heads': 8, 'head_features': 64, 'multiplier': 2}), 'dist': Munch({'sigma_data': 0.2, 'estimate_sigma_data': True, 'mean': -3.0, 'std': 1.0})})})


In [4]:
train_lines = [
    "dcd74732/wav/dcd74732_607.wav|つーかうち的には、三人が継続で立候補してくれたのがなんかもう最高じゃん?|0",
    "dcd74732/wav/dcd74732_463.wav|だからあのあとで入ったんじゃん?しおぽよの部屋で会ったときは、名前も顔も知らなかったよ?|0",
    "dcd74732/wav/dcd74732_060.wav|焼きなよー。ひょろっちい身体してるんだから、こんなときこそ焼かないでどーすんの?てゆか、あれだけ外で仕事して、なんでそんな白いの|0",
    "dcd74732/wav/dcd74732_514.wav|うちの前でカッコつけなくてもよくね?ひよりんはさておき、他の子もみんなかわいいじゃん。意識すんなってほうが無理じゃね?|0",
    "dcd74732/wav/dcd74732_464.wav|6月の頭くらいに、マネージャーさんからうちに連絡あって、直接話したんだけど…|0",
    "dcd74732/wav/dcd74732_353.wav|教頭の責任を追求するだけの行動力があればトモすけやってよ。うち応援するよ|0",
]

In [5]:
OOD_data = ["君の仮説が正しいとすれば、絶剣、君が死ねばアペイリアは恋ができなくなる|dummy",
"いやー、まぁそんな前じゃないっていうか…|dummy",
"話の流れで普通に聞いてみただけだけど?|dummy",
"まつげは長くて綺麗だし、唇はとろんとして柔らかそうだし、流し目が色っぽくて、それから…|dummy"
"時間が経てばきっと思いだすのですわ。でも今思いだしてもらわなくては、私がもう一度言わなくてはならなくなりますの。だからもう一度よく考えていただきたいですわ|dummy",
"ここはさっきまで密閉された空間だったわ。何かその…えっと、そういうのがあるかもしれないわ|dummy",]

In [8]:
%pdb off

Automatic pdb calling has been turned OFF


In [11]:
# data utils
from meldataset import build_dataloader
from utils import get_data_path_list

train_lines, _ = get_data_path_list(cfg.data_params.train_data,
                                   cfg.data_params.val_data)
dl = build_dataloader(train_lines,
                      root_path=cfg.data_params.root_path,
                      OOD_data="Data/melo/data/anime/OOD_texts.txt",
                      min_length=cfg.data_params.min_length,
                      batch_size=,
                      num_workers=0,
                      device=device,
                      collate_config={"return_wave": True})

# grab one batch
batch = next(iter(dl))
waves, texts, input_lengths, ref_texts, ref_lengths, mels, out_lens, ref_mels = batch

print("wave 0 type/shape:", type(waves[0]), waves[0].shape)
print("texts:", texts.shape, "input_lengths:", input_lengths)
print("mels:", mels.shape, "out_lens:", out_lens)

Symbol dictionary size: 174
Misaki phonemizer initialized.
99b5eb16/wav/99b5eb16_0995.wav 44100
99b5eb16/wav/99b5eb16_0995.wav 44100
7d23dc35/wav/7d23dc35_180.wav 44100
7d23dc35/wav/7d23dc35_180.wav 44100
4359417e/wav/4359417e_108.wav 44100
4359417e/wav/4359417e_108.wav 44100
31ebd5d6/wav/31ebd5d6_0196.wav 44100
31ebd5d6/wav/31ebd5d6_0196.wav 44100


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [5]:
from models import load_ASR_models, load_F0_models, build_model
from Utils.PLBERT.util import load_plbert

asr = load_ASR_models(cfg.ASR_path, cfg.ASR_config).to(device).eval()
f0  = load_F0_models(cfg.F0_path).to(device).eval()
plb = load_plbert(cfg.PLBERT_dir).to(device).eval()

In [8]:
from utils import recursive_munch
model_args = recursive_munch(cfg.model_params)

nets = build_model(model_args, asr, f0, plb)
# move to device & set eval to avoid random dropout
for name, m in nets.items():
    nets[name] = m.to(device).eval()

# inspect submodules
print(nets.keys())
print("TextEncoder:", nets.text_encoder)

dict_keys(['bert', 'bert_encoder', 'predictor', 'decoder', 'text_encoder', 'predictor_encoder', 'style_encoder', 'diffusion', 'text_aligner', 'pitch_extractor', 'mpd', 'msd', 'wd'])
TextEncoder: TextEncoder(
  (embedding): Embedding(186, 512)
  (cnn): ModuleList(
    (0-2): 3 x Sequential(
      (0): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): LayerNorm()
      (2): LeakyReLU(negative_slope=0.2)
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
)


In [7]:
nets.text_encoder.forward??

Signature: nets.text_encoder.forward(x, input_lengths, m)
Docstring:
Define the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    def forward(self, x, input_lengths, m):
        # --- Add senior's validation check ---
        # 범위를 벗어나는 인덱스 확인 및 수정
        n_symbols = self.embedding.num_embeddings
        out_of_range = (x >= n_symbols) | (x < 0)
        if out_of_range.any():
            out_of_range_count = out_of_range.sum().item()
            max_index = x.max().item()
            min_index = x.min().item()
            out_of_range_indices = out_of_range.nonzero(as_tuple=True)
            out_of_range_values = x[out_of_range_indices]
            print(f"TextEncoder: 

In [9]:
from utils import length_to_mask, log_norm, recursive_munch
from monotonic_align import mask_from_lens
from utils import maximum_path
import torch

# 1. load config & model (once)
# cfg = recursive_munch(yaml.safe_load(open("Configs/config_ja.yml")))
# nets = build_model(cfg.model_params, text_aligner, pitch_extractor, plb)
# for m in nets.values(): m.to(device).eval()

# 2. get a batch
batch = next(iter(dl))
waves, texts, input_lengths, ref_texts, ref_lengths, mels, out_lens, ref_mels = batch

# 3. move tensors to GPU
texts         = texts.to(device)         # Long [B, T_text]
input_lengths = input_lengths.to(device) # Long [B]
mels          = mels.to(device)          # Float [B, n_mels, T_mel]
ref_mels      = ref_mels.to(device)

# 4. build masks
text_mask = length_to_mask(input_lengths).to(device)          
down      = nets.text_aligner.n_down
mel_mask  = length_to_mask(out_lens // (2**down)).to(device)  # [B, T_mel_down]

95c3bdd8/wav/95c3bdd8_2275.wav 44100
95c3bdd8/wav/95c3bdd8_2275.wav 44100


In [48]:
# TextEncoder(x:Long[B,T], lengths:Long[B], mask:Bool[B,T])→Float[B,H,T]
summary(
  nets.text_encoder,
  input_data=[texts, input_lengths, text_mask],
  col_names=("input_size","output_size","num_params"),
  depth=5
)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
TextEncoder                              [1, 242]                  [1, 512, 242]             --
├─Embedding: 1-1                         [1, 242]                  [1, 242, 512]             95,232
├─ModuleList: 1-2                        --                        --                        --
│    └─Sequential: 2-1                   [1, 512, 242]             [1, 512, 242]             --
│    │    └─Conv1d: 3-1                  [1, 512, 242]             [1, 512, 242]             1,311,744
│    │    └─LayerNorm: 3-2               [1, 512, 242]             [1, 512, 242]             1,024
│    └─Sequential: 2-5                   --                        --                        (recursive)
│    │    └─LeakyReLU: 3-3               [1, 512, 242]             [1, 512, 242]             --
│    └─Sequential: 2-3                   --                        --                        (recursive)
│  

In [12]:
nets.text_encoder

TextEncoder(
  (embedding): Embedding(186, 512)
  (cnn): ModuleList(
    (0-2): 3 x Sequential(
      (0): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): LayerNorm()
      (2): LeakyReLU(negative_slope=0.2)
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
)

In [54]:
summary(
  nets.text_aligner,
  input_data=[mels, mel_mask, texts],
  col_names=("input_size","output_size","num_params"),
  depth=100
)

Layer (type:depth-idx)                             Input Shape               Output Shape              Param #
ASRCNN                                             [1, 80, 790]              [1, 395, 179]             --
├─MFCC: 1-1                                        [1, 80, 790]              [1, 40, 790]              --
├─ConvNorm: 1-2                                    [1, 40, 790]              [1, 256, 395]             --
│    └─Conv1d: 2-1                                 [1, 40, 790]              [1, 256, 395]             71,936
├─Sequential: 1-3                                  [1, 256, 395]             [1, 256, 395]             --
│    └─Sequential: 2-2                             [1, 256, 395]             [1, 256, 395]             --
│    │    └─ConvBlock: 3-1                         [1, 256, 395]             [1, 256, 395]             --
│    │    │    └─ModuleList: 4-1                   --                        --                        --
│    │    │    │    └─Sequential: 5-1

In [14]:
nets.text_aligner,

(ASRCNN(
   (to_mfcc): MFCC()
   (init_cnn): ConvNorm(
     (conv): Conv1d(40, 256, kernel_size=(7,), stride=(2,), padding=(3,))
   )
   (cnns): Sequential(
     (0): Sequential(
       (0): ConvBlock(
         (blocks): ModuleList(
           (0): Sequential(
             (0): ConvNorm(
               (conv): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
             )
             (1): ReLU()
             (2): GroupNorm(8, 256, eps=1e-05, affine=True)
             (3): Dropout(p=0.2, inplace=False)
             (4): ConvNorm(
               (conv): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
             )
             (5): ReLU()
             (6): Dropout(p=0.2, inplace=False)
           )
           (1): Sequential(
             (0): ConvNorm(
               (conv): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
             )
             (1): ReLU()
             (2): GroupNorm(8, 256, eps=1e-05, affine=True)
        

In [16]:
nets.text_aligner.__init__??

Signature:
nets.text_aligner.__init__(
    input_dim=80,
    hidden_dim=256,
    n_token=35,
    n_layers=6,
    token_embedding_dim=256,
)
Docstring: Initialize internal Module state, shared by both nn.Module and ScriptModule.
Source:   
    def __init__(self,
                 input_dim=80,
                 hidden_dim=256,
                 n_token=35,
                 n_layers=6,
                 token_embedding_dim=256,

    ):
        super().__init__()
        self.n_token = n_token
        self.n_down = 1
        self.to_mfcc = MFCC()
        self.init_cnn = ConvNorm(input_dim//2, hidden_dim, kernel_size=7, padding=3, stride=2)
        self.cnns = nn.Sequential(
            *[nn.Sequential(
                ConvBlock(hidden_dim),
                nn.GroupNorm(num_groups=1, num_channels=hidden_dim)
            ) for n in range(n_layers)])
        self.projection = ConvNorm(hidden_dim, hidden_dim // 2)
        self.ctc_linear = nn.Sequential(
            LinearNorm(hidden_dim//2, hidd

In [18]:
nets.text_aligner.forward??

Signature: nets.text_aligner.forward(x, src_key_padding_mask=None, text_input=None)
Docstring:
Define the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    def forward(self, x, src_key_padding_mask=None, text_input=None):
        x = self.to_mfcc(x)
        x = self.init_cnn(x)
        x = self.cnns(x)
        x = self.projection(x)
        x = x.transpose(1, 2)
        ctc_logit = self.ctc_linear(x)
        if text_input is not None:
            _, s2s_logit, s2s_attn = self.asr_s2s(x, src_key_padding_mask, text_input)
            return ctc_logit, s2s_logit, s2s_attn
        else:
            return ctc_logit
File:      ~/styletts2-jp-r/Utils/ASR/models.py
Type:      m

In [23]:
!uv pip install ipdb

Resolved 19 packages in 200ms                                        
Prepared 1 package in 16ms                                               
Installed 1 package in 12ms                                 
 + ipdb==0.13.13


In [26]:
ctc, s2s_l, s2s_a = nets.text_aligner(mels, mel_mask, texts)

In [30]:
plt.imshow(s2s_attn[0].cpu().detach().numpy().T, aspect="auto")

IndexError: index 1 is out of bounds for dimension 0 with size 1

> /tmp/ipykernel_2091/3277055024.py(1)<module>()
----> 1 plt.imshow(s2s_attn[1].cpu().detach().numpy().T, aspect="auto")



ipdb>  q


In [55]:
summary(
  nets.style_encoder,
  input_data=[mels.unsqueeze(1)],
  col_names=("input_size","output_size","num_params"),
  depth=100
)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
StyleEncoder                             [1, 1, 80, 790]           [1, 128]                  --
├─Sequential: 1-1                        [1, 1, 80, 790]           [1, 512, 1, 1]            --
│    └─Conv2d: 2-1                       [1, 1, 80, 790]           [1, 64, 80, 790]          640
│    └─ResBlk: 2-2                       [1, 64, 80, 790]          [1, 128, 40, 395]         111,424
│    │    └─Conv2d: 3-1                  [1, 64, 80, 790]          [1, 128, 80, 790]         8,192
│    │    └─DownSample: 3-2              [1, 128, 80, 790]         [1, 128, 40, 395]         --
│    └─ResBlk: 2-15                      --                        --                        (recursive)
│    │    └─LeakyReLU: 3-3               [1, 64, 80, 790]          [1, 64, 80, 790]          --
│    └─ResBlk: 2-6                       --                        --                        (recursive)
│    │  

In [32]:
nets.style_encoder

StyleEncoder(
  (shared): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ResBlk(
      (actv): LeakyReLU(negative_slope=0.2)
      (downsample): DownSample()
      (downsample_res): LearnedDownSample(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64)
      )
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv1x1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (2): ResBlk(
      (actv): LeakyReLU(negative_slope=0.2)
      (downsample): DownSample()
      (downsample_res): LearnedDownSample(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=128)
      )
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [56]:
nets.style_encoder.forward??

Signature: nets.style_encoder.forward(x)
Docstring:
Define the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    def forward(self, x):
        h = self.shared(x)
        h = h.view(h.size(0), -1)
        s = self.unshared(h)
    
        return s
File:      ~/styletts2-jp-r/models.py
Type:      method

In [57]:
nets.style_encoder.__init__??

Signature: nets.style_encoder.__init__(dim_in=48, style_dim=48, max_conv_dim=384)
Docstring: Initialize internal Module state, shared by both nn.Module and ScriptModule.
Source:   
    def __init__(self, dim_in=48, style_dim=48, max_conv_dim=384):
        super().__init__()
        blocks = []
        blocks += [spectral_norm(nn.Conv2d(1, dim_in, 3, 1, 1))]

        repeat_num = 4
        for _ in range(repeat_num):
            dim_out = min(dim_in*2, max_conv_dim)
            blocks += [ResBlk(dim_in, dim_out, downsample='half')]
            dim_in = dim_out

        blocks += [nn.LeakyReLU(0.2)]
        blocks += [spectral_norm(nn.Conv2d(dim_out, dim_out, 5, 1, 0))]
        blocks += [nn.AdaptiveAvgPool2d(1)]
        blocks += [nn.LeakyReLU(0.2)]
        self.shared = nn.Sequential(*blocks)

        self.unshared = nn.Linear(dim_out, style_dim)
File:      ~/styletts2-jp-r/models.py
Type:      method

In [66]:
summary(
  nets.predictor_encoder,
  input_data=[mels.unsqueeze(1)],
  col_names=("input_size","output_size","num_params"),
  depth=2
)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
StyleEncoder                             [1, 1, 80, 790]           [1, 128]                  --
├─Sequential: 1-1                        [1, 1, 80, 790]           [1, 512, 1, 1]            --
│    └─Conv2d: 2-1                       [1, 1, 80, 790]           [1, 64, 80, 790]          640
│    └─ResBlk: 2-2                       [1, 64, 80, 790]          [1, 128, 40, 395]         119,616
│    └─ResBlk: 2-15                      --                        --                        (recursive)
│    └─ResBlk: 2-6                       --                        --                        (recursive)
│    └─ResBlk: 2-15                      --                        --                        (recursive)
│    └─ResBlk: 2-6                       --                        --                        (recursive)
│    └─ResBlk: 2-7                       [1, 128, 40, 395]         [1, 256, 20, 198]     

In [67]:
nets.predictor_encoder

StyleEncoder(
  (shared): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ResBlk(
      (actv): LeakyReLU(negative_slope=0.2)
      (downsample): DownSample()
      (downsample_res): LearnedDownSample(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64)
      )
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv1x1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (2): ResBlk(
      (actv): LeakyReLU(negative_slope=0.2)
      (downsample): DownSample()
      (downsample_res): LearnedDownSample(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=128)
      )
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [69]:
nets.predictor_encoder.__init__??

Signature: nets.predictor_encoder.__init__(dim_in=48, style_dim=48, max_conv_dim=384)
Docstring: Initialize internal Module state, shared by both nn.Module and ScriptModule.
Source:   
    def __init__(self, dim_in=48, style_dim=48, max_conv_dim=384):
        super().__init__()
        blocks = []
        blocks += [spectral_norm(nn.Conv2d(1, dim_in, 3, 1, 1))]

        repeat_num = 4
        for _ in range(repeat_num):
            dim_out = min(dim_in*2, max_conv_dim)
            blocks += [ResBlk(dim_in, dim_out, downsample='half')]
            dim_in = dim_out

        blocks += [nn.LeakyReLU(0.2)]
        blocks += [spectral_norm(nn.Conv2d(dim_out, dim_out, 5, 1, 0))]
        blocks += [nn.AdaptiveAvgPool2d(1)]
        blocks += [nn.LeakyReLU(0.2)]
        self.shared = nn.Sequential(*blocks)

        self.unshared = nn.Linear(dim_out, style_dim)
File:      ~/styletts2-jp-r/models.py
Type:      method

In [70]:
nets.predictor_encoder.forward??

Signature: nets.predictor_encoder.forward(x)
Docstring:
Define the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    def forward(self, x):
        h = self.shared(x)
        h = h.view(h.size(0), -1)
        s = self.unshared(h)
    
        return s
File:      ~/styletts2-jp-r/models.py
Type:      method

In [71]:
# 5-a) text feats
t_feats = nets.text_encoder(texts, input_lengths, text_mask)

# 5-b) soft attention
_, _, attn = nets.text_aligner(mels, mel_mask, texts)
attn = attn.transpose(-1,-2)[...,1:].transpose(-1,-2)  # drop BOS

# 5-c) monotonic mask & hard alignment
from utils import length_to_mask
mel_down = out_lens // (2**down)

mask_T = length_to_mask(mel_down).to(device)      # [B, T_mel_down]
mask_S = length_to_mask(input_lengths).to(device) # [B, T_text]
mask_ST = mask_T.unsqueeze(2) | mask_S.unsqueeze(1)   # [B, T_mel_down, T_text]

attn_mono = maximum_path(attn, mask_ST)

# 5-d) style codes
s_ss  = nets.style_encoder(mels.unsqueeze(1))
s_sp  = nets.predictor_encoder(mels.unsqueeze(1))
s_cat = torch.cat([s_ss, s_sp], dim=1)  # [B, style_dim*2]

In [76]:
nets.bert.forward??

Signature: nets.bert.forward(*args, **kwargs)
Docstring:
The [`AlbertModel`] forward method, overrides the `__call__` special method.

<Tip>

Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
instance afterwards instead of this since the former takes care of running the pre and post processing steps while
the latter silently ignores them.

</Tip>

Args:
    input_ids (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
        Indices of input sequence tokens in the vocabulary.

        Indices can be obtained using [`AutoTokenizer`]. See [`PreTrainedTokenizer.__call__`] and
        [`PreTrainedTokenizer.encode`] for details.

        [What are input IDs?](../glossary#input-ids)
    attention_mask (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, *optional*):
        Mask to avoid performing attention on padding token indices. Mask values selected in `[0, 1]`:

        - 1 for tokens that are **not maske

In [83]:
nets.text_encoder

TextEncoder(
  (embedding): Embedding(186, 512)
  (cnn): ModuleList(
    (0-2): 3 x Sequential(
      (0): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): LayerNorm()
      (2): LeakyReLU(negative_slope=0.2)
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
)

In [77]:
nets.bert

CustomAlbert(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(500, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertSdpaAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0, inplace=False)
   

In [86]:
 # 1) build HF attention mask: 1 for real tokens, 0 for pad
 attn_mask = (~text_mask).long()          # shape [B, T_text], dtype=torch.long

 # 2) call your Albert/BERT
 bert_out = nets.bert(
   input_ids     = texts,                 # LongTensor
   attention_mask= attn_mask,             # LongTensor 0/1
 )
 # since your custom forward returns .last_hidden_state,
 # bert_out is actually a FloatTensor [B, T_text, hidden_size]

 print("bert_out.shape:", bert_out.shape)  # => [B, T_text, hidden_size]

 # 3) project into your TTS hidden space
 d_en = nets.bert_encoder(bert_out)        # Linear(hidden_size→hidden_dim)
 # Linear preserves batch+time dims => [B, T_text, hidden_dim]

 # 4) rearrange for conv/1d modules that expect [B, C, T]
 d_en = d_en.transpose(-1, -2)             # [B, hidden_dim, T_text]
 print("d_en.shape:", d_en.shape)

bert_out.shape: torch.Size([1, 242, 768])
d_en.shape: torch.Size([1, 512, 242])


In [89]:
   from torchinfo import summary
   # A) DurationEncoder alone
   summary(
     nets.predictor.text_encoder,       # that’s your DurationEncoder
     input_data=[d_en, s_cat, input_lengths, text_mask],
     depth=2
   )

   # B) The F0/N submodules  
   # They live in nets.predictor.F0 and nets.predictor.N (ModuleLists)
   for i, block in enumerate(nets.predictor.F0):
       summary(
         block,
         input_data=[d_en, s_cat],
         depth=1,
         col_names=("input_size","output_size","num_params")
       )

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

> /home/ryonakamura/styletts2-jp-r/.venv/lib/python3.10/site-packages/torchinfo/torchinfo.py(304)forward_pass()
    302     except Exception as e:
    303         executed_layers = [layer for layer in summary_list if layer.executed]
--> 304         raise RuntimeError(
    305             "Failed to run torchinfo. See above stack traces for more details. "
    306             f"Executed layers up to: {executed_layers}"



ipdb>  q


In [43]:
summary(
  nets.pitch_extractor,
  input_data=[mels.unsqueeze(1)],
  col_names=("input_size","output_size","num_params"),
  depth=10
)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
JDCNet                                   [1, 1, 80, 790]           [790]                     1,742,338
├─Sequential: 1-1                        [1, 1, 790, 80]           [1, 64, 790, 80]          --
│    └─Conv2d: 2-1                       [1, 1, 790, 80]           [1, 64, 790, 80]          576
│    └─BatchNorm2d: 2-2                  [1, 64, 790, 80]          [1, 64, 790, 80]          128
│    └─LeakyReLU: 2-3                    [1, 64, 790, 80]          [1, 64, 790, 80]          --
│    └─Conv2d: 2-4                       [1, 64, 790, 80]          [1, 64, 790, 80]          36,864
├─ResBlock: 1-2                          [1, 64, 790, 80]          [1, 128, 790, 40]         --
│    └─Sequential: 2-5                   [1, 64, 790, 80]          [1, 64, 790, 40]          --
│    │    └─BatchNorm2d: 3-1             [1, 64, 790, 80]          [1, 64, 790, 80]          128
│    │    └─LeakyReLU

In [37]:
nets.pitch_extractor

JDCNet(
  (conv_block): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01, inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (res_block1): ResBlock(
    (pre_conv): Sequential(
      (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    )
    (conv): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01, inplace=True)
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), 

In [45]:
# 7-a) prosody features
dur_pred, en = nets.predictor(t_feats, s_cat, input_lengths, attn_mono, text_mask)
# 7-b) F0 & N
F0_real, _, _ = nets.pitch_extractor(mels.unsqueeze(1))
N_real       = log_norm(mels.unsqueeze(1)).squeeze(1)  # [B, T_mel]
# 7-c) style code: either s_ss or sampled from diffusion
s_style = s_ss  # for reconstruction

/home/ryonakamura/styletts2-jp-r/.venv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:1136: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  result = _VF.lstm(


RuntimeError: shape '[786432, 1]' is invalid for input of size 655360

> /home/ryonakamura/styletts2-jp-r/.venv/lib/python3.10/site-packages/torch/nn/modules/rnn.py(1136)forward()
   1134             )
   1135         else:
-> 1136             result = _VF.lstm(
   1137                 input,
   1138                 batch_sizes,



ipdb>  q


In [ ]:
from utils import length_to_mask

# prepare masks
text_mask = length_to_mask(input_lengths).to(device)      # [B, T_text]
down = nets.text_aligner.n_down
mel_mask  = length_to_mask(out_lens//(2**down)).to(device)

# 1) text → encoder
t_enc = nets.text_encoder(texts.to(device), input_lengths.to(device), text_mask)

# 2) alignment & ASR-style features
with torch.no_grad():
    _, _, attn = nets.text_aligner(mels.to(device), mel_mask, texts.to(device))
attn = attn.transpose(-1,-2)[...,1:].transpose(-1,-2)   # remove BOS

# 3) compute monotonic path (optional)
from monotonic_align import mask_from_lens
from utils import maximum_path
mask_ST = mask_from_lens(attn, input_lengths, out_lens//(2**down))
attn_mono = maximum_path(attn, mask_ST)

# 4) style vectors
s_enc = nets.style_encoder(mels.unsqueeze(1).to(device))      # acoustic style
p_enc = nets.predictor_encoder(mels.unsqueeze(1).to(device))  # prosody style

# 5) prosody predictor
d_out, pros = nets.predictor(t_enc @ attn_mono, torch.cat([s_enc,p_enc],-1),
                             input_lengths.to(device), attn_mono, text_mask)

print("t_enc:",  t_enc.shape)
print("attn:",   attn.shape, "attn_mono:", attn_mono.shape)
print("style:",  s_enc.shape, p_enc.shape)
print("prosody out:", d_out.shape, pros.shape)

In [ ]:
# get F0 / norm
F0_real, _, F0 = nets.pitch_extractor(mels.unsqueeze(1).to(device))
N_real = nets.text_aligner.get_feature(mels.unsqueeze(1).to(device))
from utils import log_norm
N_real = log_norm(mels.unsqueeze(1).to(device))

# choose prosody reference
ref = torch.cat([s_enc, p_enc], dim=-1)
# decode one chunk
y_hat = nets.decoder((t_enc @ attn_mono), F0_real, N_real, s_enc)  # [B, 1, T_audio]
print("decoded waveform:", y_hat.shape)

In [ ]:
from losses import MultiResolutionSTFTLoss, GeneratorLoss, DiscriminatorLoss, WavLMLoss

stft  = MultiResolutionSTFTLoss().to(device)
gloss = GeneratorLoss(nets.mpd, nets.msd).to(device)
dloss = DiscriminatorLoss(nets.mpd, nets.msd).to(device)
wlm   = WavLMLoss(cfg.model_params.slm.model, nets.wd, 
                  cfg.preprocess_params.sr, cfg.model_params.slm.sr).to(device)

L_stft = stft(y_hat.squeeze(1), waves[0].to(device).unsqueeze(0))
L_gen  = gloss(waves[0].to(device).unsqueeze(0), y_hat)
L_dis  = dloss(waves[0].to(device).unsqueeze(0), y_hat)
L_wlm  = wlm(waves[0].to(device).unsqueeze(0), y_hat.squeeze(1))

print("STFT:", L_stft.item())
print("GenLoss:", L_gen.item())
print("DiscLoss:", L_dis.item())
print("WavLM Loss:", L_wlm.item())

In [ ]:
optim = torch.optim.Adam(nets.decoder.parameters(), lr=1e-4)  # toy optimizer

optim.zero_grad()
L_stft.backward(retain_graph=True)

# show a few gradient norms
for n,p in nets.decoder.named_parameters():
    if p.grad is not None and p.grad.norm()>0:
        print(n, p.grad.norm().item())

In [ ]:
# copy a weight before
w0 = nets.decoder.conv_first.weight.clone()
optim.step()
print("Δnorm:", (nets.decoder.conv_first.weight - w0).norm().item())